# Reverse-engineering of the surron ultrabee can bus

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import csv
import json

---
## Reading data from Files

In [27]:

with open('20240427_CAN_Trace_Motorrad_T000.csv', 'r') as f:
    reader = csv.reader(f, delimiter=";")
    data00 = list(reader)

with open('20240427_CAN_Trace_Motorrad_T001.csv', 'r') as f:
    reader = csv.reader(f, delimiter=";")
    data01 = list(reader)

data00.pop(0)
data01.pop(0)

dataCombined = data00 + data01

dataCombinedDictCsv = []
for dataPoint in dataCombined:
    dataCombinedDictCsv.append({
        "Id": dataPoint[4],
        "dlc": dataPoint[2],
        "msg": dataPoint[3],
        "time": dataPoint[0],
    })

df = pd.DataFrame(dataCombinedDictCsv)
df

In [84]:
dataCombinedDictAsc = []
with open("20240427_CAN_Trace_Motorrad_T000.asc") as f:
    content = f.readlines()
    for line in content:
        dataCombinedDictAsc.append({
            "lineId": len(dataCombinedDictAsc),
            "Id": line[101:len(line) - 1],
            "dlc": line[38:39],
            "msg": line[40:64],
            "time": line[:12],
        })

with open("20240427_CAN_Trace_Motorrad_T001.asc") as f:
    content = f.readlines()
    for line in content:
        dataCombinedDictAsc.append({
            "lineId": len(dataCombinedDictAsc),
            "Id": line[101:len(line) - 1],
            "dlc": line[38:39],
            "msg": line[40:64],
            "time": line[:12],
        })

tmpCleanedArray = []

for e in dataCombinedDictAsc:
    if (e["Id"] != '') & (e["Id"] != ' ') & (e["dlc"] != '') & (e["dlc"] != ' '):
        tmpCleanedArray.append(e)

dataCombinedDictAsc = tmpCleanedArray

for pos in range(len(dataCombinedDictAsc)):
    dataCombinedDictAsc[pos]["lineId"] = pos

df = pd.DataFrame(dataCombinedDictAsc)
df

,lineId,Id,dlc,msg,time
0,0,18,8,00 00 00 00 02 00 00 02,0.008923
1,1,258,8,25 03 00 00 00 00 00 28,0.017042
2,2,259,8,00 00 00 00 00 00 00 00,0.017292
3,3,260,8,B2 20 00 00 00 00 00 D2,0.017540
4,4,261,8,10 27 00 00 00 00 00 37,0.017788
...,...,...,...,...,...
55881,55881,259,8,FF FF 2C 04 5A 00 00 88,39.316511
55882,55882,260,8,B2 20 00 00 00 00 00 D2,39.316759
55883,55883,261,8,10 27 00 00 00 00 00 37,39.317007
55884,55884,417333248x,8,00 03 E7 03 2A 00 C0 0F,39.317296


---
## Extracting all Ids 

In [22]:
messengeIds = list(map(lambda row: (row[4]), data00))
uniqueIds = set(messengeIds)

df = pd.DataFrame(uniqueIds)
df

,0
0,274
1,275
2,417333259x
3,801
4,799
5,260
6,417333249x
7,545
8,792
9,771


---
## Massage CNT per Id

In [23]:
messagesPerId = []
for uniqueId in uniqueIds:
    var = list(map(lambda row: (row[3]), filter(lambda line: line[4] == uniqueId, dataCombined)))
    messagesPerId.append({
        "Id": uniqueId,
        "cnt": len(var),
        "msg": var
    })

df = pd.DataFrame(messagesPerId)
df

,Id,cnt,msg
0,274,1328,"[00 00 3C 37 00 00 00 00, 00 00 3C 37 00 00 00..."
1,275,1329,"[01 00 00 00 00 00 00 00, 01 00 00 00 00 00 00..."
2,417333259x,271,"[00 00 00 00 00 00 01 00, 00 00 00 00 00 00 01..."
3,801,1,[C1 00 FF FF FF FF FF FF]
4,799,27,"[00 00 00 00 00 00 00 00, 00 00 00 00 00 00 00..."
5,260,6734,"[B2 20 00 00 00 00 00 D2, B2 20 00 00 00 00 00..."
6,417333249x,136,"[60 00 2C 04 F3 01 2C 04, 60 00 2C 04 F3 01 2C..."
7,545,1307,"[00 5A 00 C5 03 00 00 54, 00 5A 00 C5 03 00 00..."
8,792,267,"[02 00 00 00 00 00 00 00, 03 00 00 00 00 00 00..."
9,771,672,"[03 03 01 02 00 00 00 00, 03 03 01 02 00 00 00..."


---
## Condense to unique messages

In [24]:
uniqueMessagesPerId = []

for message in messagesPerId:
    condensed = []

    for e in message["msg"]:
        if e not in condensed:
            condensed.append(e)

    uniqueMessagesPerId.append({
        "Id": message["Id"],
        "allCnt": message["cnt"],
        "uniqueCnt": len(condensed),
        "condensedMsg": condensed
    })

df = pd.DataFrame(uniqueMessagesPerId)
df

,Id,allCnt,uniqueCnt,condensedMsg
0,274,1328,134,"[00 00 3C 37 00 00 00 00, 00 00 40 32 00 00 00..."
1,275,1329,6,"[01 00 00 00 00 00 00 00, 01 01 00 00 00 00 00..."
2,417333259x,271,1,[00 00 00 00 00 00 01 00]
3,801,1,1,[C1 00 FF FF FF FF FF FF]
4,799,27,2,"[00 00 00 00 00 00 00 00, 00 00 00 00 00 00 00..."
5,260,6734,1,[B2 20 00 00 00 00 00 D2]
6,417333249x,136,3,"[60 00 2C 04 F3 01 2C 04, 5F 00 2C 04 F3 01 2C..."
7,545,1307,5,"[00 5A 00 C5 03 00 00 54, 01 5A 00 C5 03 00 00..."
8,792,267,10,"[02 00 00 00 00 00 00 00, 03 00 00 00 00 00 00..."
9,771,672,1,[03 03 01 02 00 00 00 00]


---
## Extracting specific ids

In [25]:
data00Filtered = list(filter(lambda row: (row[4] == "417333248x"), data00))

df = pd.DataFrame(data00Filtered)
df

,0,1,2,3,4
0,0.018079,Rx,8,00 07 E7 03 9C 0F C2 0F,417333248x
1,0.043026,Rx,8,00 07 E7 03 AD 0F C5 0F,417333248x
2,0.058161,Rx,8,00 07 E7 03 C4 0F C5 0F,417333248x
3,0.078231,Rx,8,00 07 E7 03 CE 0F C2 0F,417333248x
4,0.098286,Rx,8,00 07 E7 03 E0 0F C2 0F,417333248x
...,...,...,...,...,...
4806,106.479.729,Rx,8,00 03 E7 03 2A 00 C0 0F,417333248x
4807,106.499.987,Rx,8,00 03 E7 03 28 00 C0 0F,417333248x
4808,106.519.796,Rx,8,00 03 E7 03 28 00 C0 0F,417333248x
4809,106.540.856,Rx,8,00 03 E7 03 28 00 C0 0F,417333248x


---
## Plot over time

In [26]:
searchedId = "785"
dataByte = 6

# searchedId = "417333248x"
# dataByte = 6

plotData = map(lambda row: ({"time": row["time"], "val": int(row["msg"].split()[dataByte], 16)}),
               list(filter(lambda item: item['Id'] == searchedId, dataCombinedDictCsv)))

# df = pd.DataFrame(plotData).plot.line("time", "val")
df = pd.DataFrame(plotData)
df


,time,val
0,0.081907,203
1,0.195897,203
2,0.309897,203
3,0.423894,203
4,0.538404,203
...,...,...
1322,38.867.084,203
1323,38.968.081,203
1324,39.069.085,203
1325,39.170.089,203
